# Agent state occupancy & trajectories

## Init

In [1]:
from pathlib import Path
import numpy as np

In [2]:
save_path = Path("..") / "save"
save_path.absolute()

PosixPath('/home/kir0ul/Projects/RL_Olfaction/cartesian_polar/../save')

In [85]:
sessions_path = save_path / "2025-01-19_16-28-35_EastWest_scratch"
sessions_path.absolute()

PosixPath('/home/kir0ul/Projects/RL_Olfaction/cartesian_polar/../save/2025-01-19_16-28-35_EastWest_scratch')

In [86]:
data_path = sessions_path / "data.npz"
data_path.absolute()

PosixPath('/home/kir0ul/Projects/RL_Olfaction/cartesian_polar/../save/2025-01-19_16-28-35_EastWest_scratch/data.npz')

In [87]:
data_path.exists()

False

In [14]:
data = np.load(data_path, allow_pickle=True)

In [15]:
[item for item in data.keys()]

['rewards',
 'steps',
 'episodes',
 'all_states',
 'all_actions',
 'losses',
 'p',
 'epsilons',
 'weights',
 'biases',
 'weights_val_stats',
 'biases_val_stats',
 'weights_grad_stats',
 'biases_grad_stats',
 'net',
 'env']

## State occupancy

In [16]:
data["all_states"].shape

(20550, 19)

In [17]:
data["all_actions"].shape

(20550,)

In [ ]:
def plot_locations_count(data, title, scale=False, cols=5, rows=5):
    # Preprocess
    hist2d, xedges, yedges = np.histogram2d(x=data.x, y=data.y, bins=[cols, rows])
    hist2d_res = hist2d.T
    if scale:
        locations_scaled = minmax_scale(hist2d.T.flatten())
        locations_scaled = locations_scaled.reshape((cols, rows))
        hist2d_res = locations_scaled

    cmap = sns.color_palette("rocket_r", as_cmap=True)
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(hist2d_res, cmap=cmap, ax=ax)
    ax.set_title(title, pad=20)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()
    return hist2d_res

## Trajectories